LEARNING LANGCHAIN BY USING LANGCHAIN (PYTHON)
===============================================

BY: ELLIOTT RISCH

DATE: 2023-10-11

In [ ]:
# Before you begin, you will need to create a .env file in the same directory as this notebook. 
# The .env file should contain the following lines, with your own api keys filled in:

OPENAI_API_KEY="sk-XXXXXX"

PINECONE_ENV="xx-xxxxx-xxx-free"

PINECONE_API_KEY="xxxxx-xxxx-xxxx-xxxx-xxxxx"

# You should also have installed the required packages in requirements.txt. 

***I. SETTING UP AND USING --> ChatOpenAI***

**Step 1:** Find and load the .env file containing the relevant api keys.

In [1]:
from dotenv import load_dotenv, find_dotenv # for .env file
load_dotenv(find_dotenv()) # load environment variables from .env file

# Should read "True" below, if the .env file was loaded correctly.

True

**STEP 2:** Run a quick test query to assure that the OpenAI API key has been loaded correctly from the .env.

In [3]:
from langchain.llms import OpenAI # Import the model class
llm = OpenAI(model_name="text-davinci-003") # Create a new model instance
llm("explain large language models in one sentence") # Call the model with a prompt

# See the system's response below:

'\n\nLarge language models use large amounts of data to learn the probability of a word given its context in a given language.'

**STEP 3:** Call the class imports necessary to talk with a OpenAI chat model, i.e., GPT-3 or GPT-4.

In [5]:
from langchain.schema import( # "Schema" here means the mode of communication between the human and the system.
    AIMessage,  # What the system says to you. 
    HumanMessage, # What you use to pose your prompt to the system.
    SystemMessage, # What you use to tell the system it's role.
)
from langchain.chat_models import ChatOpenAI # From langchain's chat_models module, we will import the ChatOpenAI class to go with the chosen schema.

**STEP 4:** Create a new instance of the OpenAI chat model class, create a list of the components of the chat, and then call the chat function to generate a response.

In [6]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.3) # Create a new chat instance
messages = [ # Create a list of messages to send to the system
        SystemMessage(content="You are an expert data scientist."), # Tell the system it's role
        HumanMessage(content="Write a python script that trains a neural network on simulated data.") # Pose a prompt to the system
]
response = chat(messages) # Send the messages to the system

**STEP 4:** Display the system's response.

In [7]:
print(response.content,end='\n') # Print the system's response

Sure! Here's an example of a Python script that trains a neural network on simulated data using the Keras library:

```python
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Generate simulated data
np.random.seed(0)
X = np.random.rand(1000, 10)
y = np.random.randint(2, size=(1000, 1))

# Define the model architecture
model = Sequential()
model.add(Dense(32, input_dim=10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X, y)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")
```

In this script, we first generate simulated data using `numpy`. We create a random matrix `X` of shape `(1000, 10)` and a random binary vector `y` of shape `(1000, 1)`.

Then, we define the neural network model using `Sequential` from Keras

***II. SETTING UP AND USING --> PromptTemplates***

**STEP 1:** Call the class imports necessary to talk with a OpenAI chat model and to use the PromptTemplates class.

In [8]:
from langchain.llms import OpenAI # Import the model class
from langchain import PromptTemplate # Import the PromptTemplate class, which allows you to create prompts with variables.

template = """
You are an expert data scientist with an expertise in building deep learning models.
Explain the concept {concept} in a couple of sentences.
""" # Create a template for a prompt, with a variable "concept" that can be filled in later.

prompt = PromptTemplate( # Create a new prompt instance
    input_variables=["concept"], # Specify the variables that will be filled in
    template=template, # Specify the template to use
)

**STEP 2:** Print the prompt template for the chat model.

In [9]:
prompt # Print the prompt

PromptTemplate(input_variables=['concept'], output_parser=None, partial_variables={}, template='\nYou are an expert data scientist with an expertise in building deep learning models.\nExplain the concept {concept} in a couple of sentences.\n', template_format='f-string', validate_template=True)

**STEP 3:** Define the llm and send the prompt template to the chat model with a variable value.

In [10]:
llm = OpenAI(model_name="text-davinci-003") # Create a new model instance
llm(prompt.format(concept="gradient descent")) # Call the model with the prompt, with the variable filled in

'\nGradient descent is an optimization algorithm used to find the values of parameters (coefficients) of a function that minimizes a cost function. It works by iteratively computing the partial derivatives of the cost function with respect to the parameters and updating the parameters in the negative direction of the gradient of the cost function with respect to the parameters.'

***III. SETTING UP AND USING --> Chains***

**STEP 1:** Make a chain call to the chat model using the prompt template that we have already defined.

In [11]:
from langchain.chains import LLMChain # Import the LLMChain class, which allows you to chain together multiple language model calls.

chain = LLMChain( # Create a new chain instance
    llm=llm, # Specify the language model to use
    prompt=prompt, # Specify the prompt to use
)

# run the chain only specifying the input variable.
print(chain.run("gradient descent")) # Call the chain with the input variable



Gradient descent is an optimization algorithm used to find the set of parameters that minimize a given cost function. It works by iteratively updating the parameters in the direction of the negative gradient of the cost function with respect to the parameters. The magnitude of the updates is determined by a parameter called the learning rate.


**STEP 2:** Add a new link to the chain.

In [13]:
template = "Turn this({ml_concept}) concept description into an explanation that a five year old child would understand." # Specify the template

second_prompt = PromptTemplate( # Create a new prompt instance
    input_variables=["ml_concept"], # Specify the variables that will be filled in
    template=template, # Specify the template to use
)   

chain_two = LLMChain( # Create a new chain instance 
    llm=llm, # Specify the language model to use
    prompt=second_prompt, # Specify the prompt to use
)

**STEP 3:** Combine the two links of the chain into a single chain.

In [14]:
from langchain.chains import SimpleSequentialChain # Import the SimpleSequentialChain class, which allows you to chain together multiple language model calls.

sequential_chain = SimpleSequentialChain( # Create a new chain instance
    chains=[chain, chain_two], # Specify the chains to use
    verbose=True, # Specify whether to print the intermediate results
)

# run the chain only specifying the input variable for the first chain.
explanation = sequential_chain.run("gradient descent") # Call the chain with the input variable



> Entering new SimpleSequentialChain chain...

Gradient descent is an iterative optimization algorithm used to find the values of parameters (such as weights and biases) that minimize a given cost function. It works by iteratively making small adjustments to the parameters in the direction of the negative gradient of the cost function with respect to the parameters.


Gradient descent is like a game where you try to find the best way down a mountain. You start at the top and try different paths until you find the one that takes you to the bottom in the least amount of steps.

> Finished chain.
